In [2]:
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
import scipy
from scipy import signal

# ENCODING

In [3]:
def getTextToTransmit():
    with open("input.txt", "r") as f:
        text_to_transmit = f.read()
    return text_to_transmit;

def encodeText( text ):
    #each character is a vector of 8 bits
    return np.unpackbits(bytearray(text,"utf-8"))

def convoluteEncoding(bits):
    b=[1,1]+list(map(lambda x:(x-0.5)*2,bits))
    x=[xi for j in range(2,len(b)) for xi in [b[j]*b[j-2],b[j]*b[j-1]*b[j-2]]]
    return x

def scaleArrayKTimes(k,bits):
    return [b for b in bits for i in range(k)]

def addCalibrationZeros(k,bits):
    return np.concatenate((np.zeros(k),bits))

def writeChannelInput(bits):
    with open("encoded.txt", "w") as f:
        stringToSend = "\n".join(map(lambda bit:str(bit),bits))
        #stringToSend = stringToSend[:-1]
        f.write(stringToSend)

In [4]:
MAX_SIZE=51200
NB_BYTES=80
BITS_PER_BYTES=8
ENCODER_FACTOR=2
BITS_ARRAY_SIZE=NB_BYTES*BITS_PER_BYTES*ENCODER_FACTOR
print("BITS_ARRAY_SIZE :",BITS_ARRAY_SIZE)

REPEAT_FACTOR= MAX_SIZE//(BITS_ARRAY_SIZE)-1
print("REPEAT_FACTOR :",REPEAT_FACTOR)

CALIBRATION_SIZE=BITS_ARRAY_SIZE
print('BITS_ARRAY_SIZE * REPEAT_FACTOR + CALIBRATION_SIZE =',BITS_ARRAY_SIZE,'*',REPEAT_FACTOR,'+',CALIBRATION_SIZE,'=',BITS_ARRAY_SIZE*REPEAT_FACTOR+CALIBRATION_SIZE)

x=addCalibrationZeros(CALIBRATION_SIZE,scaleArrayKTimes(REPEAT_FACTOR,convoluteEncoding(encodeText(getTextToTransmit()))))


print("input length :",len(x),"maximum is",MAX_SIZE)

BITS_ARRAY_SIZE : 1280
REPEAT_FACTOR : 39
BITS_ARRAY_SIZE * REPEAT_FACTOR + CALIBRATION_SIZE = 1280 * 39 + 1280 = 51200
input length : 51200 maximum is 51200


In [5]:
writeChannelInput(x)

# DECODING

In [6]:
def getChannelOutput():
    with open("received.txt", "r") as f:
        bitArray = np.array(f.read().split('\n'))
        return bitArray[:-1];

In [11]:
with open("received.txt", "r") as f:
        bitArray = np.array(f.read().split('\n'))
len(bitArray)

49921

In [7]:
y=getChannelOutput()

In [9]:
y[:CALIBRATION_SIZE]

array(['-4.053290386999969241e-01', '-1.617656878529849518e+00',
       '-1.324486304203899900e+00', ..., '1.994170104133754062e+00',
       '3.368159707502949018e+00', '-5.848820805454565175e+00'],
      dtype='<U25')

In [8]:
biais=np.mean(y[:CALIBRATION_SIZE])

TypeError: cannot perform reduce with flexible type